# ResearchShortcut: ArXiv Paper Quick Scanner

### 1. Instalar dependências
Execute (ou adapte) o comando abaixo em uma célula de **terminal** ou **notebook**:
```bash
pip install langchain-core langchain-community langchain-huggingface huggingface-hub arxiv pandas ipywidgets
```

> **Obs.** Todas essas bibliotecas são gratuitas e não exigem chave da OpenAI.

In [11]:
#!pip install transformers sentencepiece accelerate torch --upgrade


### 2. Configurar chave gratuita da Hugging Face
1. Crie uma conta gratuita em https://huggingface.co.
2. Gere um *Access Token* (Profile ▸ Settings ▸ Access Tokens).
3. Defina a variável de ambiente antes de rodar o notebook:
```python
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "SEU_TOKEN_AQUI"
```
Isso é **opcional** para modelos open‑source hospedados pela Hugging Face, mas previne limites estritos de requisições anônimas.

In [1]:
import os, pandas as pd, arxiv
from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe_small = pipeline("text2text-generation",
                      model="google/flan-t5-small",
                      max_new_tokens=96,
                      device_map="auto")
llm = HuggingFacePipeline(pipeline=pipe_small)


summary_prompt = PromptTemplate.from_template(
    '''Você é um assistente que ajuda pesquisadores.
    **Instrução**: Para o artigo abaixo, gere um resumo curto (máx 3 linhas) e depois uma lista de até 3 contribuições‑chave em bullets.
    Saída no formato:
    Resumo: <texto>
    Contribuições:
    - ...
    - ...
    - ...

    Artigo:
    {title}
    {abstract}'''
)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

c:\Users\marco\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marco\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu
C:\Users\marco\AppData\Local\Temp\ipykernel_13556\2014655971.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe_small)


In [2]:
def search_arxiv(query: str, max_results: int = 3):
    """Retorna lista de objetos arxiv.Result."""
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
    )
    return list(search.results())


In [3]:
def brief_paper(paper):
    text = summary_prompt.format(title=paper.title, abstract=paper.summary)
    return llm.invoke(text)


In [4]:
def run(query: str, n: int = 3):
    """Busca n artigos e retorna DataFrame resumido."""
    results = []
    for paper in search_arxiv(query, n):
        summary = brief_paper(paper)
        results.append({
            'Título': paper.title,
            'Autores': ', '.join(a.name for a in paper.authors),
            'Resumo & Contribuições': summary,
            'PDF': paper.pdf_url,
            'Publicado': paper.published.date(),
        })
    return pd.DataFrame(results)


In [8]:
df = run('large language model evaluation', 4)
df

C:\Users\marco\AppData\Local\Temp\ipykernel_13556\2161051291.py:8: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  return list(search.results())


,Título,Autores,Resumo & Contribuições,PDF,Publicado
0,Modeling Cosmic-Ray Transport: Magnetized vers...,"Jeremiah Lübke, Patrick Reichherzer, Sophie Ae...",Our results highlight the critical role of coh...,http://arxiv.org/pdf/2505.18155v1,2025-05-23
1,The Staircase of Ethics: Probing LLM Value Pri...,"Ya Wu, Qiang Sheng, Danding Wang, Guang Yang, ...","We introduce the Multi-step Moral Dilemmas, th...",http://arxiv.org/pdf/2505.18154v1,2025-05-23
2,REN: Fast and Efficient Region Encodings from ...,"Savya Khosla, Sethuraman TV, Barnett Lee, Alex...","We introduce the Region Encoder Network (REN),...",http://arxiv.org/pdf/2505.18153v1,2025-05-23
3,"Fann or Flop: A Multigenre, Multiera Benchmark...","Wafa Alghallabi, Ritesh Thawkar, Sara Ghaboura...","Fann or Flop, the first benchmark designed to ...",http://arxiv.org/pdf/2505.18152v1,2025-05-23


### 3. Próximos passos sugeridos
- **Melhorar prompt** para controlar tamanho do texto.
- Trocar modelo por outro gratuito (ex.: `mistralai/Mistral-7B-Instruct-v0.2`) ou rodar localmente via `Ollama`.
- Adicionar classificação de tags automáticas (ex.: via embeddings + k‑means).
- Integrar UI simples em Streamlit ou Gradio.